In [1]:
import datetime as dt
from lxml import html
from bs4 import BeautifulSoup
import urllib
import sys
import requests
import operator
from datetime import timedelta
import csv
import pandas as pd
import numpy as np
import math

In [8]:
year1617 = pd.read_csv('season1617.csv', parse_dates=['Date'])
year1718 = pd.read_csv('season1718.csv', parse_dates=['Date'])
year1819 = pd.read_csv('season1819.csv', parse_dates=['Date'])
year1920 = pd.read_csv('season1920.csv', parse_dates=['Date'])

In [9]:
frames = [year1617,year1718,year1819,year1920]

In [10]:
allyears = pd.concat(frames)
allyears = allyears.drop_duplicates(subset=['Date', 'Home'],keep='last')
# full = allyears[['Home','Away','WinnerHome','KenPomVal_Away','BPIval_Away','SRS_Away','KenPomVal_Home','BPIval_Home','SRS_Home']]


In [11]:
allyears

,Unnamed: 0,Date,Win_Score,Lose_Score,Home,Away,WinnerHome,KenPomRank_Away,KenPomVal_Away,BPIrank_Away,...,d_eff_Home,ppg_Home,papg_Home,Pace_Home,Three_Par_Home,True_Shoot_Home,Tov_Home,Opp_Three_Par_Home,Opp_True_Shoot_Home,Opp_Tov_Home
0,0,2016-11-11,94,49,Duke,Marist,1,312.0,-13.63,305.0,...,91.17,80.8,70.2,69.9,0.383,0.588,14.2,0.288,0.510,14.9
1,1,2016-11-11,87,64,Kentucky,Stephen F. Austin,1,233.0,-6.19,262.0,...,86.87,84.9,71.5,74.3,0.316,0.564,13.4,0.333,0.507,16.9
2,2,2016-11-11,103,99,Kansas,Indiana,0,44.0,14.49,32.0,...,89.65,83.2,71.9,71.4,0.359,0.583,15.1,0.342,0.515,16.1
3,3,2016-11-11,88,48,Villanova,Lafayette,1,330.0,-16.79,330.0,...,87.45,77.2,62.7,65.6,0.433,0.612,15.0,0.400,0.506,17.6
4,4,2016-11-11,91,77,Oregon,Army,1,230.0,-6.04,253.0,...,88.20,78.9,65.8,68.8,0.388,0.582,14.7,0.378,0.494,16.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5298,5298,2020-03-11,74,69,Alabama-Birmingham,Texas-San Antonio,1,210.0,-3.84,230.0,...,99.65,13.0,67.3,67.2,0.300,0.537,19.0,0.409,0.514,16.2
5299,5299,2020-03-11,71,69,Oregon State,Utah,1,114.0,5.56,117.0,...,111.51,13.0,71.2,66.6,0.349,0.559,15.3,0.364,0.529,16.2
5300,5300,2020-03-11,86,73,Arkansas,Vanderbilt,1,169.0,0.14,165.0,...,109.40,12.0,75.8,71.4,0.399,0.553,14.4,0.333,0.516,19.2
5301,5301,2020-03-11,77,70,Arizona,Washington,1,54.0,12.67,70.0,...,113.66,11.0,76.4,70.3,0.347,0.546,14.5,0.377,0.498,18.1


In [14]:
full = allyears.dropna()

In [15]:
full

,Unnamed: 0,Date,Win_Score,Lose_Score,Home,Away,WinnerHome,KenPomRank_Away,KenPomVal_Away,BPIrank_Away,...,d_eff_Home,ppg_Home,papg_Home,Pace_Home,Three_Par_Home,True_Shoot_Home,Tov_Home,Opp_Three_Par_Home,Opp_True_Shoot_Home,Opp_Tov_Home
0,0,2019-11-05,69,62,Kentucky,Michigan State,1,7.0,24.03,4.0,...,113.37,6.0,74.4,68.5,0.266,0.561,15.8,0.383,0.489,15.1
1,1,2019-11-05,68,66,Duke,Kansas,1,1.0,30.23,1.0,...,117.58,6.0,82.5,72.9,0.315,0.559,15.2,0.267,0.489,17.7
2,2,2019-11-05,87,74,Miami (FL),Louisville,0,9.0,21.36,8.0,...,107.91,16.0,71.2,68.7,0.372,0.525,15.1,0.347,0.526,15.1
3,3,2019-11-05,74,59,Florida,North Florida,1,166.0,0.37,151.0,...,113.49,12.0,71.9,66.3,0.368,0.555,15.5,0.356,0.519,16.9
4,4,2019-11-05,95,71,Maryland,Holy Cross,1,344.0,-21.34,345.0,...,114.29,7.0,71.8,67.7,0.431,0.532,14.3,0.381,0.489,15.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5298,5298,2020-03-11,74,69,Alabama-Birmingham,Texas-San Antonio,1,210.0,-3.84,230.0,...,99.65,13.0,67.3,67.2,0.300,0.537,19.0,0.409,0.514,16.2
5299,5299,2020-03-11,71,69,Oregon State,Utah,1,114.0,5.56,117.0,...,111.51,13.0,71.2,66.6,0.349,0.559,15.3,0.364,0.529,16.2
5300,5300,2020-03-11,86,73,Arkansas,Vanderbilt,1,169.0,0.14,165.0,...,109.40,12.0,75.8,71.4,0.399,0.553,14.4,0.333,0.516,19.2
5301,5301,2020-03-11,77,70,Arizona,Washington,1,54.0,12.67,70.0,...,113.66,11.0,76.4,70.3,0.347,0.546,14.5,0.377,0.498,18.1


In [17]:
y = 0
for x in full.columns:
    print(x,y)
    y+=1

Unnamed: 0 0
Date 1
Win_Score 2
Lose_Score 3
Home 4
Away 5
WinnerHome 6
KenPomRank_Away 7
KenPomVal_Away 8
BPIrank_Away 9
BPIval_Away 10
SRS_Away 11
SOS_Away 12
o_eff_Away 13
d_eff_Away 14
ppg_Away 15
papg_Away 16
Pace_Away 17
Three_Par_Away 18
True_Shoot_Away 19
Tov_Away 20
Opp_Three_Par_Away 21
Opp_True_Shoot_Away 22
Opp_Tov_Away 23
KenPomRank_Home 24
KenPomVal_Home 25
BPIrank_Home 26
BPIval_Home 27
SRS_Home 28
SOS_Home 29
o_eff_Home 30
d_eff_Home 31
ppg_Home 32
papg_Home 33
Pace_Home 34
Three_Par_Home 35
True_Shoot_Home 36
Tov_Home 37
Opp_Three_Par_Home 38
Opp_True_Shoot_Home 39
Opp_Tov_Home 40


In [43]:
X = full.iloc[:,[12,18,19,20,21,22,23,29,35,36,37,38,39,40]]
y = full.iloc[:,6]

In [44]:
y

0       1
1       1
2       0
3       1
4       1
       ..
5298    1
5299    1
5300    1
5301    1
5302    0
Name: WinnerHome, Length: 5067, dtype: int64

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=14)

In [46]:
LR = LogisticRegression().fit(X_train, y_train)

c:\users\kobelb\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [47]:
LR.score(X_test, y_test)

0.7080867850098619

In [48]:
LR.coef_ , LR.intercept_, LR.n_iter_

(array([[-0.12224181, -0.82798116, -5.43912021,  0.16353032,  0.51680932,
          6.25521172, -0.10385319,  0.12370001, -1.23272323,  6.15031956,
         -0.17148755, -1.18638871, -5.65317292,  0.11257787]]),
 array([0.82284549]),
 array([100]))

In [49]:
from sklearn import metrics
y_pred = LR.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[200, 379],
       [ 65, 877]], dtype=int64)

In [50]:
def conf(game):
    arr = LR.coef_
    print("Coefficients:",arr)
    print("Team Stats:",game)
    coefmult = arr*game
    print("Multiplied:",coefmult)
    comb_coef = coefmult.sum()+LR.intercept_
    print("Sum of multiplied:",comb_coef)
    es = math.exp(comb_coef)
    print("E to the sum of multiplied:",es)
    confidence = es / (1+es)
    print("conf:", confidence)

In [53]:
ISUatKU = [11.63,.381,.538,15.9,.377,.535,18.1,10.07,.332,.568,16.2,.414,.462,17.1]
predict = [ISUatKU]
res = LR.predict_proba(predict)
confidence = conf(ISUatKU)
print()
print("Prob from Model",res)

Coefficients: [[-0.12224181 -0.82798116 -5.43912021  0.16353032  0.51680932  6.25521172
  -0.10385319  0.12370001 -1.23272323  6.15031956 -0.17148755 -1.18638871
  -5.65317292  0.11257787]]
Team Stats: [11.63, 0.381, 0.538, 15.9, 0.377, 0.535, 18.1, 10.07, 0.332, 0.568, 16.2, 0.414, 0.462, 17.1]
Multiplied: [[-1.42167228 -0.31546082 -2.92624667  2.60013215  0.19483711  3.34653827
  -1.87974265  1.24565906 -0.40926411  3.49338151 -2.77809833 -0.49116492
  -2.61176589  1.92508163]]
Sum of multiplied: [0.79505954]
E to the sum of multiplied: 2.214572848554193
conf: 0.6889166781677489

Prob from Model [[0.31108332 0.68891668]]


In [54]:
import pandas as pd
feature_imp = pd.Series(LR.coef_[0],index=X.columns).sort_values(ascending=False)
feature_imp

Opp_True_Shoot_Away    6.255212
True_Shoot_Home        6.150320
Opp_Three_Par_Away     0.516809
Tov_Away               0.163530
SOS_Home               0.123700
Opp_Tov_Home           0.112578
Opp_Tov_Away          -0.103853
SOS_Away              -0.122242
Tov_Home              -0.171488
Three_Par_Away        -0.827981
Opp_Three_Par_Home    -1.186389
Three_Par_Home        -1.232723
True_Shoot_Away       -5.439120
Opp_True_Shoot_Home   -5.653173
dtype: float64